In [3]:
# ! pip install piexif

In [14]:
import os
import piexif

from tqdm.notebook import tqdm
from PIL import Image

from joblib import Parallel, delayed

from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

from pathlib import Path

import pandas as pd

In [10]:
images_to_rescale = [file for file in Path("gabon_extra_data/chimpanzee/").glob("*")] + \
                    [file for file in Path("gabon_extra_data/gorilla/").glob("*")]
len(images_to_rescale)

1219

In [12]:
sizes = [Image.open(file).size for file in images_to_rescale]
sizes[:10]

[(2048, 1536),
 (1920, 1080),
 (1920, 1080),
 (1920, 1080),
 (1920, 1080),
 (2048, 1536),
 (2048, 1536),
 (2048, 1536),
 (2048, 1536),
 (2048, 1536)]

In [19]:
images_x_y = pd.DataFrame()
images_x_y["uniqueName"] = images_to_rescale
images_x_y["x"] = [size[0] for size in sizes]
images_x_y["y"] = [size[1] for size in sizes]

images_x_y[:4]

,uniqueName,x,y
0,gabon_extra_data/chimpanzee/D0526142.JPG,2048,1536
1,gabon_extra_data/chimpanzee/H1848415.JPG,1920,1080
2,gabon_extra_data/chimpanzee/J1445261.JPG,1920,1080
3,gabon_extra_data/chimpanzee/I1048731.JPG,1920,1080


In [48]:
# PATH_TO_IMG_RESC = Path("/data_rescaled/")
PATH_TO_IMG_RESC = Path("gabon_extra_data/rescaled_extra")

In [53]:
exceptions = []
for i in tqdm(range(len(images_x_y))):
    image_x_y = images_x_y.iloc[i,:]
    try:
        filename = Path(image_x_y[0]).name
        new_filename = "resc_" + filename
        if os.path.isfile(PATH_TO_IMG_RESC / new_filename):
            continue
        img = Image.open(image_x_y[0])
        y = 384
        scale = 384 / int(image_x_y[2])
        x = round(scale * int(image_x_y[1]))
        img = img.resize((x, y), Image.ANTIALIAS)
        img.save(PATH_TO_IMG_RESC / new_filename)
        piexif.transplant(str(image_x_y[0]), PATH_TO_IMG_RESC / new_filename)
    except Exception as e:
        exceptions.append([image_x_y, e])

In [56]:
len(exceptions)

1211

In [59]:
df = pd.read_csv("gabon_extra_data/train_valid_df_200722.csv")
df

,uniqueName,species,main_folder,is_valid,sub_folder
0,0000005.jpg,Human,Compressed Camera Trap Images,False,T33
1,0000007.jpg,Human,Compressed Camera Trap Images,False,T33
2,0000009.jpg,Human,Compressed Camera Trap Images,False,T33
3,0000013.jpg,Human,Compressed Camera Trap Images,False,T33
4,0000015.jpg,Human,Compressed Camera Trap Images,False,T33
...,...,...,...,...,...
348149,K0758088.JPG,Gorilla,extra_data,False,not_specified
348150,F2451519.JPG,Gorilla,extra_data,False,not_specified
348151,J2559677.JPG,Gorilla,extra_data,False,not_specified
348152,I2831434.JPG,Gorilla,extra_data,False,not_specified


In [61]:
len(set(df.uniqueName).intersection(set([file.name for file in images_to_rescale])))

1213

In [62]:
len(set([file.name for file in images_to_rescale]))

1216